In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import pathlib

In [2]:
#test_images 路徑設定
path_test="theSimpsons-test/test"
data_dir_test = pathlib.Path(path_test)

In [3]:
#Number of test
image_count = len(list(data_dir_test.glob('*/*.jpg')))
print(image_count)

10791


In [4]:
#設定圖片大小與分堆
batch_size = 16
img_height = 180
img_width = 180

datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

#讀取test_images
test_ds = datagen.flow_from_directory(
  data_dir_test,
  target_size=(img_height, img_width),
  shuffle=False, seed=None,
  batch_size=batch_size)

Found 10791 images belonging to 1 classes.


In [5]:
class_names = test_ds.class_indices
print(class_names)

{'non': 0}


In [6]:
#設定暫存
#AUTOTUNE = tf.data.AUTOTUNE

#test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [7]:
import keras
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate,Dropout
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet import ResNet50

In [8]:
#載入訓練好的模型 
model = keras.models.load_model('lab-logs/models/Best-model-7.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 6, 6, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 50)                25650     
Total params: 26,236,338
Trainable params: 26,183,218
Non-trainable params: 53,120
_________________________________________________________________


In [9]:
#進行預測
predict=model.predict(test_ds)

In [10]:
#找出預測機率最大
preds_classes = np.argmax(predict, axis=-1)

In [11]:
#預測結果
preds_classes

array([17, 25, 39, ..., 29, 30, 36], dtype=int64)

In [12]:
#取得類別
path="theSimpsons-train/train"
data_dir = pathlib.Path(path)

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

Found 96929 files belonging to 50 classes.
Using 77544 files for training.
['abraham_grampa_simpson', 'agnes_skinner', 'apu_nahasapeemapetilon', 'barney_gumble', 'bart_simpson', 'brandine_spuckler', 'carl_carlson', 'charles_montgomery_burns', 'chief_wiggum', 'cletus_spuckler', 'comic_book_guy', 'disco_stu', 'dolph_starbeam', 'duff_man', 'edna_krabappel', 'fat_tony', 'gary_chalmers', 'gil', 'groundskeeper_willie', 'homer_simpson', 'jimbo_jones', 'kearney_zzyzwicz', 'kent_brockman', 'krusty_the_clown', 'lenny_leonard', 'lionel_hutz', 'lisa_simpson', 'lunchlady_doris', 'maggie_simpson', 'marge_simpson', 'martin_prince', 'mayor_quimby', 'milhouse_van_houten', 'miss_hoover', 'moe_szyslak', 'ned_flanders', 'nelson_muntz', 'otto_mann', 'patty_bouvier', 'principal_skinner', 'professor_john_frink', 'rainier_wolfcastle', 'ralph_wiggum', 'selma_bouvier', 'sideshow_bob', 'sideshow_mel', 'snake_jailbird', 'timothy_lovejoy', 'troy_mcclure', 'waylon_smithers']


In [13]:
#轉換成類別名稱
predict_name=[]
for i in preds_classes:
    predict_name.append(class_names[i])

In [14]:
import pandas as pd
df=pd.read_csv("submission.csv")
df.id=df.id.astype("str")
#排序 in alphanumeric order
df=df.sort_values(by="id")
df

,id,character
0,1,gil
9,10,gil
99,100,gil
999,1000,gil
9999,10000,gil
...,...,...
9994,9995,gil
9995,9996,gil
9996,9997,gil
9997,9998,gil


In [15]:
#將預測結果 存入df
df.character=predict_name
df

,id,character
0,1,gil
9,10,lionel_hutz
99,100,principal_skinner
999,1000,marge_simpson
9999,10000,moe_szyslak
...,...,...
9994,9995,kent_brockman
9995,9996,lisa_simpson
9996,9997,marge_simpson
9997,9998,martin_prince


In [16]:
#再重新排序 id
df.id=df.id.astype("int16")
df.sort_values(by="id")

,id,character
0,1,gil
1,2,disco_stu
2,3,patty_bouvier
3,4,charles_montgomery_burns
4,5,lisa_simpson
...,...,...
10786,10787,chief_wiggum
10787,10788,krusty_the_clown
10788,10789,bart_simpson
10789,10790,jimbo_jones


In [17]:
#輸出test的預測檔案
df.to_csv('submit_5th.csv',index=False)